In [1]:
# This notebook uses the RAGAS evaluation library to evaluate several metrics for a RAG pipeline
# I use the Google Gemini API (free tier, local API key), but RAGAS is compatible with several LLMs

# Google Gemini: https://ai.google.dev/gemini-api/docs/models/gemini
# RAGAS: https://docs.ragas.io/en/stable/
# I used RAGAS v0.1.15 (mine was dev version from Github, but 0.1.15 from PyPI should work fine). 

# Note: I had to edit underlying RAGAS library (cloned locally, edited files, then pip -e installed locally) for this issue re: temperature with Gemini:
# https://github.com/explodinggradients/ragas/pull/657/files
# https://github.com/explodinggradients/ragas/issues/678
# Edits simply remove the temperature variable from the relevant source files; see notes.txt for more specific info

# RAGAS had a large redesign from version 0.1 to 0.2 : https://docs.ragas.io/en/stable/howtos/migrations/migrate_from_v01_to_v02/
# - Note that on RAGAS version 0.2.6, the temperature edits did not make RAGAS compatible with Gemini.
# - There are open Github issues about Gemini's compatibility for RAGAS version 2+; more time and edits would be needed.

In [2]:
# RAGAS metrics guide: https://docs.ragas.io/en/latest/concepts/metrics/index.html#ragas-metrics

# Faithfulness - Measures the factual consistency of the answer to the context based on the question.
# Context_precision - Measures how relevant the retrieved context is to the question, conveying the quality of the retrieval pipeline.
# Answer_relevancy - Measures how relevant the answer is to the question.
# Context_recall - Measures the retriever’s ability to retrieve all necessary information required to answer the question.

# Faithfulness with HHEM - Similar to Faithfulness but uses a HuggingFace model (Vectara's HHEM 2.1 classifier) to detect hallucinations
# https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html#faithfullness-with-hhem-2-1-model
# https://huggingface.co/vectara/hallucination_evaluation_model

# RAGAS has other metrics as well : https://docs.ragas.io/en/latest/concepts/metrics/index.html

In [1]:
# Set do not track variable for RAGAS
# More info: https://github.com/explodinggradients/ragas/issues/49
import os
os.environ["RAGAS_DO_NOT_TRACK"] = "True"

In [2]:
import logging
import sys
import textwrap
import ast
import time
import pandas as pd
from IPython.display import display
from IPython.display import Markdown

# Replace these two Google Gemini imports with imports for your LLM
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

from langchain_community.document_loaders import DirectoryLoader
from llama_index.core import Document, VectorStoreIndex, Settings, StorageContext, load_index_from_storage
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

import ragas
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas.run_config import RunConfig
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas import evaluate
from datasets import Dataset

In [3]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
# Double check the RAGAS do not track setting
ragas._analytics.do_not_track()

True

In [5]:
# Set up local API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [ ]:
# Establish RAG pipeline with Gemini

In [7]:
# # Create a Faiss vector store for RAG
# # If you already have an index created, skip a few coding cells to the LLM / embeddings setup

# # Example of creating a small vector store
# # Using 4 State of the Union speeches, all text from whitehouse.gov briefing room speeches posted online, edited to include a title with the date of the speech
# # Example from 2024:
# # https://www.whitehouse.gov/briefing-room/speeches-remarks/2024/03/07/remarks-of-president-joe-biden-state-of-the-union-address-as-prepared-for-delivery-2/

# # load and parse files
# sotu = []
# newfiles = ["./Speeches/titleedits/state_of_the_union_042921.txt", "./Speeches/titleedits/state_of_the_union_030122.txt", "./Speeches/titleedits/state_of_the_union_020723.txt", "./Speeches/titleedits/state_of_the_union_030724.txt"]
# for i in newfiles:
#     with open(i) as file:
#         for line in file:
#             nl = line.rstrip()
#             if nl != '':
#                 sotu.append(nl)

# # convert into Document format
# documents = [Document(text=line) for line in sotu]

In [9]:
# # Example of a loaded Document line
# documents[-1]

Document(id_='235d1f3b-a216-412c-8459-51d27c73c8d0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='May God protect our troops.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [ ]:
# # Set up the faiss index
# d = 768 # dimensions of the input vector of the embedding model that we're going to use; in this case, the google embedding model
# faiss_index = faiss.IndexFlatL2(d)
# print(faiss_index.is_trained) # double check that the training worked

In [6]:
# Set up the llm, embeddings, and Settings for Faiss 
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash") # Replace with your LLM
doc_embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004") # Replace with your embeddings model
Settings.embed_model = doc_embeddings # used for LlamaIndex FaissVectorStore
Settings.llm = llm # used for LlamaIndex FaissVectorStore

In [7]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [15]:
# # Uncomment for when you need to re-embed and vectorize documents

# vector_store = FaissVectorStore(faiss_index=faiss_index)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context, show_progress=True
# )

# # Save index to disk
# index.storage_context.persist()

# # Save/remember index id for loading next time
# index.index_id

In [8]:
# After you have a saved index, load that index for RAG answer generation:

# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage"
)
# My local index id '3d3c99c5-aa1c-42d7-a9ce-c4bb12fbc6d5' uses the 4 speeches including a title that includes the date it was given
index = load_index_from_storage(storage_context=storage_context, index_id='3d3c99c5-aa1c-42d7-a9ce-c4bb12fbc6d5')

INFO:root:Loading llama_index.vector_stores.faiss.base from ./storage/default__vector_store.json.
Loading llama_index.vector_stores.faiss.base from ./storage/default__vector_store.json.
INFO:llama_index.core.indices.loading:Loading indices with ids: ['3d3c99c5-aa1c-42d7-a9ce-c4bb12fbc6d5']
Loading indices with ids: ['3d3c99c5-aa1c-42d7-a9ce-c4bb12fbc6d5']


In [11]:
# # Optional- if you'd like to query your index
# # Set up query and chat engines with the index
# query_engine = index.as_query_engine(similarity_top_k=10)
# chat_engine = index.as_chat_engine(similarity_top_k=10, chat_mode='context')

In [ ]:
# # Example query and response with Gemini and query_engine
# query = "What has the President done related to healthcare?"
# response = query_engine.query(query) 
# print(response.response)

In [ ]:
# # Get ranked scores for top k RAG source nodes
# for node in response.source_nodes:
#     print(f"{node.get_score()} -> {node.text}")

In [ ]:
# # Example of using the chat engine with our index
# query = "You are an expert speech analyst and specialize in analyzing Presidential State of the Union speeches. Could you please analyze the speeches and generate 2 questions and answers from each speech, providing the document filename of each speech that relates to each question?"
# response = chat_engine.chat(query) 
# print(response.response)

In [22]:
# # Optional: View chat history
# chat_engine.chat_history

In [20]:
# Code for RAGAS evaluation library to work with Gemini and our local RAG setup

In [21]:
# Example of generating synthetic dataset with RAGAS

# In a synthetic dataset, columns generated are 'question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', and 'episode_done'
# Ground truth is supposed to be the 'human' level answer vs the RAG answer

# Notes: 
# - We have to generate the answer separately with our RAG, which then generates new context used.
# - I use the context that was used to generate the answer for the metrics calculation, while still saving the old contexts column.
# - The best thing to do would be to generate the answer when creating the synthetic test dataset, but this is not available in RAGAS.
# - From a Github issue: Since you use the same LLM to generate your synthetic dataset ground_truth and your answer, 
# - it is possible the results of the RAG evaluation might be biased. This has not been studied.

In [9]:
# Load documents for use in generating synthetic dataset with RAGAS
loader = DirectoryLoader("./Speeches/titleedits") # Loads all docs in the directory; there are parameters for ignoring or matching certain files
documents = loader.load()

In [11]:
# Need to add 'filename' metadata for RAGAS to process documents
for document in documents:
    document.metadata['filename'] = document.metadata['source']

In [38]:
# Synthetic testset generator with Gemini models
generator_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", timeout=240) # Other notable parameters: temperature=0.7, transport="rest"
critic_llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", timeout=240) 
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", request_options={"timeout": 240}) 

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

In [39]:
# Increase the timeout settings with RAGAS's RunConfig class

# Note: For Gemini, the RAGAS internal RunConfig settings do a decent job at limiting the 429 resource exhausted warnings
# (max_workers=1 still can send more requests to Gemini than the 15 requests per minute it allows)
# Still very difficult to have the testset generation run successfully with Gemini free tier
# I also tried the ratelimit and backoff libraries in Python, but I still got so many 429 warnings that the generation failed
# Sometimes even the 1 max worker will not finish, but it will finish occasionally

run_config = RunConfig(timeout=240, max_retries=20, max_wait=240, max_workers=1)

In [21]:
# Generate the synthetic dataset/testset
def generate_testset_rate(docs):
    """
    Calls the LLM and embeddings model to generate the synthetic dataset with rate limit run_config
    Can change the distribution of simple, reasoning, and multi-context questions generated
    """
    testset = generator.generate_with_langchain_docs(documents=documents, # LangChain source documents
                                                     test_size=50, # number of test samples to generate
                                                     distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25}, 
                                                     is_async=False,
                                                     raise_exceptions=True, 
                                                     run_config=run_config)
    return testset

In [ ]:
testset = generate_testset_rate(documents=documents, run_config=run_config)  # edited to include run_config, ToDo: Test this
testset_pd = testset.to_pandas()

In [34]:
# # Save generated testset to csv 
# testset_pd.to_csv('datasets/testset_flash_pro15.csv', index=False)

In [23]:
# Now generate answers for the testset, as answers are not automatically generated at creation

# testset_pd = pd.read_csv("datasets/testset_flash_pro15.csv", index_col = None)

# Note: When saving, the 'contexts' column is saved as a string but needs to be a list
# If you are importing testset_pd from a csv file, use the below code to change the column to a list

# testset_pd['contexts'] = testset_pd['contexts'].apply(ast.literal_eval)

In [ ]:
# Generate answers
query_engine = index.as_query_engine(similarity_top_k=10)
answers = [query_engine.query(q) for q in testset_pd['question']]

In [25]:
# Parse out new 'answer' and 'contexts' columns
answers_new = []
context_new = []
for i in answers:
    answers_new.append(i.response)
    context_new.append([c.node.get_content() for c in i.source_nodes])

testset_pd = testset_pd.rename(columns={"contexts":"contexts_gt"}) # Keeping old contexts that were used for testset/query generation (gt = ground truth)
testset_pd['contexts'] = context_new
testset_pd['answer'] = answers_new

# Save complete synthetically created dataset/testset
# testset_pd.to_csv('datasets/ragas_full_testset_flash_pro15.csv', index=False)

In [ ]:
# Evaluate a dataset with RAGAS

In [12]:
# Read in dataset for evaluation
testset_pd = pd.read_csv("datasets/unlabeled_dataset/unlabeled_dataset.csv", index_col = None) 

# RAGAS expects the following columns (rename in dataset as needed) : "question", "answer", "ground_truth", "source_file", "contexts"
# testset_pd = testset_pd.rename(columns={"Query": "question", "Answer": "answer", "Expected_Output": "ground_truth", "Contexts": "orig_contexts", "Source_File":"source_file", "Document": "contexts"})

# Note: When saving, the 'contexts' column is saved as a string but needs to be a list
# If you are importing testset_pd from a csv file, use the below code to change the column to a list
# testset_pd['contexts'] = testset_pd['contexts'].apply(ast.literal_eval)

In [16]:
testset_pd

,question,answer,ground_truth,orig_contexts,source_file,contexts
0,Identify specific examples of government inves...,The transcontinental railroad and the intersta...,The speech highlights several examples: the tr...,; discovering vaccines; gave us the Internet a...,Speeches/titleedits/state_of_the_union_042921.txt,"[Throughout our history, if you think about it..."
1,"Does the American Jobs Plan, a large-scale inv...",The plan seeks to create jobs by modernizing i...,The American Jobs Plan aims to create jobs by ...,; discovering vaccines; gave us the Internet a...,Speeches/titleedits/state_of_the_union_042921.txt,[The American Jobs Plan creates jobs replacing...
2,Considering the significant impact of cancer o...,Investing in cancer research is a priority bec...,Investing in cancer research is a priority bec...,"But so many of us have deceased sons, daughter...",Speeches/titleedits/state_of_the_union_042921.txt,"[But so many of us have deceased sons, daughte..."
3,How does the President's viewpoint on infrastr...,The President believes that infrastructure inv...,The President emphasizes that infrastructure i...,"But so many of us have deceased sons, daughter...",Speeches/titleedits/state_of_the_union_042921.txt,"[Investments in jobs and infrastructure, like ..."
4,Analyze the potential economic consequences of...,"A progressive tax structure, where higher earn...",The President advocates for raising taxes on c...,you should be able to become a billionaire an...,Speeches/titleedits/state_of_the_union_042921.txt,[When you hear someone say that they don’t wan...
...,...,...,...,...,...,...
795,Explain how the president's call for aid to Ga...,The president emphasizes the importance of inc...,The president's call for aid to Gaza is direct...,I say we must stop it. \n\nI’m proud we beat ...,Speeches/titleedits/state_of_the_union_030724.txt,"[Tonight, I’m directing the U.S. military to l..."
796,"If a ceasefire were to fail, how would the Pre...",The President's proposed humanitarian efforts ...,The President's proposal for a temporary pier ...,I say we must stop it. \n\nI’m proud we beat ...,Speeches/titleedits/state_of_the_union_030724.txt,"[Tonight, I’m directing the U.S. military to l..."
797,What steps is the US President taking to achie...,The US President is directing the military to ...,The US President is working to achieve a cease...,I say we must stop it. \n\nI’m proud we beat ...,Speeches/titleedits/state_of_the_union_030724.txt,"[Tonight, I’m directing the U.S. military to l..."
798,Identify the central theme of the President's ...,The President strongly advocates for reproduct...,The President's opening remarks regarding repr...,no place in America! \n\nHistory is watching....,Speeches/titleedits/state_of_the_union_030724.txt,"[Like most Americans, I believe Roe v. Wade go..."


In [17]:
# At least for RAGAS v0.1.15, we need to convert the pandas testset into Dataset format for the evaluate function to work
# Note: I am also dropping the original contexts column here
testset_ds = Dataset.from_pandas(testset_pd.drop("orig_contexts", axis=1))

In [18]:
testset_ds

Dataset({
    features: ['question', 'answer', 'ground_truth', 'source_file', 'contexts'],
    num_rows: 800
})

In [41]:
# Note: I'm using the normal LLM, not the RAG context-loaded query engine
# There is code at the bottom of the notebook for using the query engine, which should be the way to go
# However, that code appears to be broken from RAGAS right now, so I was forced to use the regular Gemini LLM

# Note: The RAGAS evaluate function (below) may re-run the query and give new answers and contexts
# See this issue: https://github.com/explodinggradients/ragas/issues/1211
# In testing, the results still give me the same answers and contexts as I started with, so I'm not concerned by this

ragas_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", timeout=600) # try request_timeout=120
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [40]:
# Increase the timeout settings
run_config = RunConfig(timeout=300, max_wait=3000, max_workers=1, max_retries=10)  # Increase timeout to 180 seconds

In [ ]:
# Optional parameter: 
# in_ci: bool, Whether the evaluation is running in CI or not. 
# If set to True then some metrics will be run to increase the reproducability of the evaluations. 
# This will increase the runtime and cost of evaluations. Default is False.
# In practice, setting in_ci = True resulted in a lot of timeouts / no score calculated / NaN
evalresult = evaluate(
    metrics = [
        context_precision
        #faithfulness,
        #answer_relevancy,
        #context_recall
    ],
    dataset = testset_ds,
    llm = ragas_llm,
    embeddings=embeddings,
    run_config=run_config
)

In [31]:
evalresult # original batch run

{'context_precision': 0.3405}

In [34]:
evalresult.to_pandas().to_csv("results/evalresult.csv") # original batch run

In [2]:
evalresults = pd.read_csv("results/evalresult.csv")

In [5]:
evalresults['context_precision'].isna().sum()

533

In [ ]:
# Not documented that I could find, discovered in github issue
# evalresult.to_pandas() will give sample-level scores!

In [61]:
# Example result:
# {'context_precision': 0.4676, 'faithfulness': 1.0000, 'answer_relevancy': 0.6515, 'context_recall': 0.8000}
# Reran
# {'context_precision': 0.5979, 'faithfulness': 1.0000, 'answer_relevancy': 0.6533, 'context_recall': 0.8000}
evalresult

{'context_precision': 0.5979, 'faithfulness': 1.0000, 'answer_relevancy': 0.6533, 'context_recall': 0.8000}

In [26]:
tempdataset = testset_ds.select(range(0,1))

In [21]:
testset_results = pd.DataFrame()

In [22]:
scores = []

In [45]:
# ToDo: let this run (in batches) over the entire dataset next time if evaluate call on entire dataset doesn't work
# Optional parameter: 
# in_ci: bool, Whether the evaluation is running in CI or not. 
# If set to True then some metrics will be run to increase the reproducibility of the evaluations. 
# This will increase the runtime and cost of evaluations. Default is False.
# In practice, setting in_ci = True resulted in a lot of timeouts / no score calculated / NaN
# Stopped 12/2 0-147, 12/3: 147-297, 12/4: 297-447, 12/5: 448-598, 12/6: 447-448, 598-747
for i in range(740,747):
    tempdataset = testset_ds.select(range(i, i+1))
    print(i)
    evalresult = evaluate(
        metrics = [
            context_precision
            #faithfulness,
            #answer_relevancy,
            #context_recall
        ],
        dataset = tempdataset,
        llm = ragas_llm,
        embeddings=embeddings,
        run_config=run_config
    )
    scores.append(evalresult['context_precision'])
    testset_results = pd.concat([testset_results, evalresult.to_pandas()])
    time.sleep(60)

740


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

741


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

742


...

745


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

746


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


In [51]:
evalresult.to_pandas()

,question,answer,ground_truth,source_file,contexts,context_precision
0,Compare the American Jobs Plan with the Americ...,The American Jobs Plan focuses on creating job...,The American Jobs Plan focuses on infrastructu...,Speeches/titleedits/state_of_the_union_042921.txt,"[Look, the American Jobs Plan will help millio...",0.0


In [48]:
#testset_results = pd.concat([testset_results, evalresult.to_pandas()])
pd.DataFrame(scores, index=None).mean()

0    0.359384
dtype: float64

In [49]:
len(testset_results)

150

In [50]:
testset_results

,question,answer,ground_truth,source_file,contexts,context_precision
0,Compare the author's position on cutting Socia...,The author strongly opposes any cuts to Social...,The author firmly opposes any cuts to Social S...,Speeches/titleedits/state_of_the_union_020723.txt,[Instead of making the wealthy pay their fair ...,0.770833
0,"Considering the reduction of the deficit, the ...",The administration has achieved significant re...,The administration has achieved significant pr...,Speeches/titleedits/state_of_the_union_020723.txt,"[In the last two years, my administration cut ...",0.833333
0,Is it fair that the administration is proposin...,The administration believes it is fair to incr...,The President believes that the wealthy and co...,Speeches/titleedits/state_of_the_union_020723.txt,[And we pay for these investments in our futur...,0.809524
0,Analyze the rhetorical strategies employed by ...,The speaker uses powerful rhetoric to galvaniz...,The speaker employs several rhetorical strateg...,Speeches/titleedits/state_of_the_union_030724.txt,[My fellow Americans the issue facing our nati...,0.000000
0,"How might these policies, if implemented, impa...",The policies described aim to shift the distri...,"These policies, if implemented, could have a s...",Speeches/titleedits/state_of_the_union_030724.txt,[America’s comeback is building a future of Am...,0.000000
...,...,...,...,...,...,...
0,What was the deadliest day for the Jewish peop...,Empty Response,"October 7th, when Hamas slaughtered 1,200 inno...",Speeches/titleedits/state_of_the_union_030724.txt,[The deadliest day for the Jewish people since...,0.750000
0,Compare the US President's efforts to establis...,The US President is actively working to establ...,The US President is working to establish an im...,Speeches/titleedits/state_of_the_union_030724.txt,[Israel has an added burden because Hamas hide...,0.000000
0,How has the speaker addressed the affordabilit...,The speaker has addressed the affordability of...,The speaker has addressed the affordability of...,Speeches/titleedits/state_of_the_union_030724.txt,[Let’s continue increasing Pell Grants for wor...,1.000000
0,Summarize the speaker's proposals for making t...,The speaker proposes several measures to make ...,The speaker proposes raising the minimum tax r...,Speeches/titleedits/state_of_the_union_030724.txt,[The way to make the tax code fair is to make ...,0.000000


In [51]:
testset_results.to_csv("results/results_ragas_unlabeled_smallruns_5.csv", index=False)

In [50]:
tempdataset = testset_ds.select(range(446, 449))

In [51]:
tempdataset['question']

['Analyze the political motivations behind GOP proposals regarding Medicare and Social Security sunset provisions.',
 "Compare the author's position on cutting Social Security and Medicare to the views of some Republicans, highlighting their differing opinions on these programs and their impact on the American public.",
 "Compare the tone and themes of President Biden's State of the Union Address to that of previous addresses, focusing on his approach to bipartisanship and domestic issues."]

In [ ]:
# Note: received warning for the answer where there was no response from the llm, definitely reduced faithfulness score

# Results:
# Using contexts generated when produced answers from LLM:
# new testset_answer_newcontext_flash_pro15.csv result, with contexts_gt (aka contexts generated with ground truth generation) column removed
# {'context_precision': 0.4171, 'faithfulness': 0.9167, 'answer_relevancy': 0.6509, 'context_recall': 0.8000}
# reran
# {'context_precision': 0.4676, 'faithfulness': 1.0000, 'answer_relevancy': 0.6515, 'context_recall': 0.8000}
# reran
# {'context_precision': 0.5979, 'faithfulness': 1.0000, 'answer_relevancy': 0.6533, 'context_recall': 0.8000}

# Compared to using contexts generated for ground truth (probably not correct):
# new testset_answer_newcontext_flash_pro15.csv result, using old contexts
# {'context_precision': 0.7500, 'faithfulness': 0.7392, 'answer_relevancy': 0.6041, 'context_recall': 1.0000}
# reran:
# {'context_precision': 0.8500, 'faithfulness': 0.7123, 'answer_relevancy': 0.5934, 'context_recall': 1.0000}
# reran:
# {'context_precision': 0.7500, 'faithfulness': 0.6556, 'answer_relevancy': 0.5638, 'context_recall': 1.0000}

In [ ]:
# Evaluation results on metrics:

# RAGAS metrics guide: https://docs.ragas.io/en/latest/concepts/metrics/index.html#ragas-metrics
# I don't have example ranges to compare anything to, so below is my best guess.

# Faithfulness - Measures the factual consistency of the answer to the context based on the question.
# 0.9167 - 1.0000 indicates that the LLM is staying true to the facts provided in the context for answering the question.
# There is another Faithfulness metric: from ragas.metrics import FaithulnesswithHHEM
# This uses a huggingface model to help detect hallucination : https://huggingface.co/vectara/hallucination_evaluation_model
# See below for code : {'faithfulness_with_hhem': 0.6319} 
# This doesn't really agree with the RAGAS faithfulness score... may need to dive in further another time.
# Context_precision - Measures how relevant the retrieved context is to the question, conveying the quality of the retrieval pipeline.
# At 0.4171 - 0.5979, suggests that the context isn't particularly relevant to the question.
# Answer_relevancy - Measures how relevant the answer is to the question.
# 0.6509 - 0.6533 seems moderately low, just going off of the number.
# Context_recall - Measures the retriever’s ability to retrieve all necessary information required to answer the question.
# 0.8 indicates that the llm context is decently good and can typically answer the question or most of it. 

In [105]:
# Test run, just compare to using contexts_gt column instead of the newer context generated with the answer
testset_ds_oldcontext = Dataset.from_pandas(testset_pd.drop("contexts", axis=1).rename(columns={'contexts_old':'contexts'}))

In [ ]:
evalresult_old2 = evaluate(
    testset_ds_oldcontext,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm = ragas_llm,
    embeddings=embeddings, 
    run_config=run_config
)

In [ ]:
# new testset_answer_newcontext_flash_pro15.csv result, using old contexts
# {'context_precision': 0.7500, 'faithfulness': 0.7392, 'answer_relevancy': 0.6041, 'context_recall': 1.0000}
# reran:
# {'context_precision': 0.8500, 'faithfulness': 0.7123, 'answer_relevancy': 0.5934, 'context_recall': 1.0000}
# reran:
# {'context_precision': 0.7500, 'faithfulness': 0.6556, 'answer_relevancy': 0.5638, 'context_recall': 1.0000}
evalresult_old2

In [ ]:
# RAGAS also has an additional Faithfulness with HHEM metric (yes- it is misspelled in their documentation) 
# that uses a HuggingFace model to detect hallucinations
# Note: There's a message on HuggingFace about the token indices sequence length error being normal and an artifact; thus, ignoring the below error
# https://huggingface.co/vectara/hallucination_evaluation_model
from ragas.metrics import FaithulnesswithHHEM
faithfulness_with_hhem = FaithulnesswithHHEM()
result_faithfulness_hhem = evaluate(
    testset_ds,
    metrics=[faithfulness_with_hhem],
    llm = ragas_llm,
    embeddings=embeddings, 
    run_config=run_config
)

In [158]:
# with context from answer generation:
# {'faithfulness_with_hhem': 0.6319}
# testing: with context from ground truth/synthetic testset generation
# {'faithfulness_with_hhem': 0.5241}
# this seems to agree with the RAGAS faithfulness score in that answers seem to be partially made up.
result_faithfulness_hhem

{'faithfulness_with_hhem': 0.6319}

In [84]:
# Extra non-working code:

In [85]:
# Code to use the query_engine in the evaluation 
# Modeled after this tutorial: https://docs.ragas.io/en/latest/howtos/applications/compare_llms.html

# Does not currently work: for some metrics, it is not finding the 'ground_truth' column in the dataset
# For other metrics, appears to run but returns the below errors and returns 'nan' for results

In [64]:
# start of testing to try and get rag query engine for evaluate
def generate_responses(query_engine, test_questions, test_answers):
  responses = [query_engine.query(q) for q in test_questions]

  answers = []
  contexts = []
  for r in responses:
    answers.append(r.response)
    contexts.append([c.node.get_content() for c in r.source_nodes])
  dataset_dict = {
        "question": test_questions,
        "answer": answers,
        "contexts": contexts,
  }
  if test_answers is not None:
    dataset_dict["ground_truth"] = test_answers
  ds = Dataset.from_dict(dataset_dict)
  return ds

test_questions = testset_pd['question'].values.tolist()
test_answers = [[item] for item in testset_pd['answer'].values.tolist()]

result_ds = generate_responses(query_engine, test_questions, test_answers)

In [ ]:
# Note: This evaluate function that uses the query_engine does not return results (nan for all metrics)
# Errors (below are repeated many times):
# WARNING:ragas.llms.base:n values greater than 1 not support for LlamaIndex LLMs
# n values greater than 1 not support for LlamaIndex LLMs
# INFO:ragas.llms.base:callbacks not supported for LlamaIndex LLMs, ignoring callbacks
# callbacks not supported for LlamaIndex LLMs, ignoring callbacks
# ERROR:ragas.executor:Exception raised in Job[5]: TimeoutError()
# Exception raised in Job[5]: TimeoutError()
# ERROR:ragas.executor:Exception raised in Job[19]: AttributeError('ChatGoogleGenerativeAI' object has no attribute 'acomplete')
# Exception raised in Job[19]: AttributeError('ChatGoogleGenerativeAI' object has no attribute 'acomplete')

from ragas.integrations.llama_index import evaluate

eval_qe2 = evaluate(
    query_engine=query_engine,
    dataset=result_ds,
    metrics=[faithfulness,
    answer_relevancy,
    context_utilization],
    llm=ragas_llm,
    embeddings=embeddings, 
    run_config=run_config
)

In [63]:
eval_qe2

{'faithfulness': nan, 'answer_relevancy': nan, 'context_utilization': nan}